In [1]:
import os
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import zscore
from scipy.stats import iqr
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests

In [31]:
project_dir = '/Users/xiaoqianxiao/projects/NARSAD'
for phase in [2,3]:
    event_file = os.path.join(project_dir,f'MRI/source_data/behav/task-Narsad_phase{phase}_events.csv')
    df_event = pd.read_csv(event_file)
    df_tmp = df_event.copy()
    df_tmp = df_tmp.sort_values(by=['trial_type', 'onset'])  # Sort by 'trial_type' and 'onset'
    df_tmp['half_index'] = df_tmp.groupby('trial_type').cumcount()
    half_count = df_tmp.groupby('trial_type')['onset'].transform('count') // 2
    df_tmp.loc[df_tmp['trial_type'] != 'SHOCK', 'trial_type'] = df_tmp.apply(
        lambda row: f"{row['trial_type']}_first_half" if row['half_index'] < half_count[row.name] 
        else f"{row['trial_type']}_second_half",
        axis=1
    )
    df_tmp = df_tmp.drop(columns=['half_index'])
    #print(df_tmp)
    results_file = os.path.join(project_dir, f'MRI/source_data/behav/task-Narsad_phase{phase}_half_events.csv')
    df_tmp.to_csv(results_file, sep="\t", index=False)

In [32]:
event_file = os.path.join(project_dir,f'MRI/source_data/behav/task-Narsad_phase-3_sub-202_events.csv')
df_event = pd.read_csv(event_file)
df_tmp = df_event.copy()
df_tmp = df_tmp.sort_values(by=['trial_type', 'onset'])  # Sort by 'trial_type' and 'onset'
df_tmp['half_index'] = df_tmp.groupby('trial_type').cumcount()
half_count = df_tmp.groupby('trial_type')['onset'].transform('count') // 2
df_tmp.loc[df_tmp['trial_type'] != 'SHOCK', 'trial_type'] = df_tmp.apply(
    lambda row: f"{row['trial_type']}_first_half" if row['half_index'] < half_count[row.name] 
    else f"{row['trial_type']}_second_half",
    axis=1
)
df_tmp = df_tmp.drop(columns=['half_index'])
#print(df_tmp)
results_file = os.path.join(project_dir, f'MRI/source_data/behav/task-Narsad_phase-3_sub-202_half_events.csv')
df_tmp.to_csv(results_file, sep="\t", index=False)

In [27]:
df_tmp.sort_values(by=['trial_type','onset'])

,onset,duration,trial_type
4,40,6,CS-_first_half
16,154,6,CS-_first_half
24,230,6,CS-_first_half
30,289,6,CS-_first_half
32,308,6,CS-_second_half
36,347,6,CS-_second_half
40,386,6,CS-_second_half
46,441,6,CS-_second_half
8,79,6,CSR_first_half
12,114,6,CSR_first_half


In [36]:
regress_data = pd.read_csv(results_file, sep=r'\s+')

In [39]:
regress_data

,onset,duration,trial_type
0,40,6,CS-_first_half
1,154,6,CS-_first_half
2,230,6,CS-_first_half
3,289,6,CS-_first_half
4,308,6,CS-_second_half
5,347,6,CS-_second_half
6,386,6,CS-_second_half
7,441,6,CS-_second_half
8,59,6,CSR_first_half
9,96,6,CSR_first_half


In [44]:
root_dir = '/Users/xiaoqianxiao/projects'
project_name = 'NARSAD'
data_dir = os.path.join(root_dir, project_name, 'MRI')
behav_dir = os.path.join(data_dir, 'source_data/behav')
task = 'phase2'
regressors_file = os.path.join(behav_dir, 'task-Narsad_'+task+'_half_events.csv')

In [45]:
regress_data = pd.read_csv(regressors_file, sep='\t')

In [46]:
regress_data

,onset,duration,trial_type
0,12,6,CS-_first_half
1,107,6,CS-_first_half
2,147,6,CS-_first_half
3,166,6,CS-_first_half
4,241,6,CS-_second_half
...,...,...,...
58,265,0,US_CSS_first_half
59,305,0,US_CSS_second_half
60,324,0,US_CSS_second_half
61,361,0,US_CSS_second_half
